In [10]:
%display latex

### Setup


In [8]:
import numpy as np
s1 = matrix(
    [
        [0, 1],
        [1, 0]
    ]
)
s2 = matrix(
    [
        [0, -i],
        [i, 0]
    ]
)
s3 = matrix(
    [
        [1, 0],
        [0, -1]
    ]
)

one = matrix.identity(s1.dimensions()[0])

s = [s1, s2, s3]
s

[
[0 1]  [ 0 -I]  [ 1  0]
[1 0], [ I  0], [ 0 -1]
]

In [12]:
# Hack
# Needed to take conjugate of pi functions
# Do not know of a better way

def Id(self, x): return self(x)

p = vector([
    function("pi"+str(i), latex_name="\\pi_"+str(i), conjugate_func=Id)(x) for i  in range(1, 4)
])
p

(pi1(x), pi2(x), pi3(x))

In [16]:
Q = 1/6 * one + 1/2*s3
Q

[ 2/3    0]
[   0 -1/3]

In [17]:
var("e", latex_name="\\varepsilon", domain="real")
var("d", latex_name="\\delta", domain="real")
var("mu", latex_name="\\mu_I", domain="real")
var("a", latex_name="\\alpha", domain="real")

pi_s = e * sum([s[i]*p[i] for i in range(len(s))])
v = 1/2*d*mu*s3

n = 3 # expand up to, not incluing, n

Functions for simplifying and printing expressions

In [18]:
def mat_series(mat, x, n):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].series(x, n).truncate()

def mat_simp(mat):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].full_simplify()
            mat[i, j] = mat[i, j].trig_reduce()

def mat_prep(mat, n=n):
    mat_series(mat, e, n)
    mat_simp(mat)
    return mat

def print_e(elem):
    coeff = elem.coefficients(e)
    for i in range(len(coeff)):
        print(e^coeff[i][1], ":")
        c = coeff[i][0].coefficients(mu)
        for k in c:
            s = k[0].full_simplify()*mu**k[1]
            pretty_print(s.factor())

def print_e2(elem):
    coeff = elem.coefficients(e)
    for i in range(len(coeff)):
        print(e^coeff[i][1], ":")
        c = coeff[i][0].coefficients(mu)
        for k in c:
            s = k[0].full_simplify()*mu**k[1]
            pretty_print(s.factor().full_simplify())

In [19]:
acom = lambda A1, A2 : A1 * A2 + A2 * A1
com = lambda A1, A2 : A1 * A2 - A2 * A1

def proj(A):
    A1 = A.trace()/2
    A = A - one * A1
    v = [A1, acom(A, s1)[0, 0]/2, acom(A, s2)[0, 0]/2, acom(A, s3)[0, 0]/2]
    v = [a.full_simplify().trig_reduce() for a in v]
    return v 

In [20]:
POW = lambda A, n : matrix.identity(A.dimensions()[0]) if (n == 0) else A * POW(A, n-1)
EXP = lambda A, n : sum([POW(A, i)/factorial(i) for i in range(n+1)])

### Find $\Sigma$ and derivatives

In [30]:
A_a = one*cos(a/2) + I * s1 * sin(a/2)
U = lambda n: EXP(I * pi_s/2, n)
SIGMA = lambda n: mat_prep(A_a * U(n) * U(n) * A_a)
S = mat_prep(SIGMA(n), n=n)

S = mat_prep(U(n)*U(n), n=n)
proj(S)

[-1/2*e^2*pi1(x)^2 - 1/2*e^2*pi2(x)^2 - 1/2*e^2*pi3(x)^2 + 1,
 I*e*pi1(x),
 I*e*pi2(x),
 I*e*pi3(x)]

In [34]:
(mat_prep(Q*S*Q*S.C.T, n=n)).trace().simplify_full()

-e^2*pi1(x)^2 - e^2*pi2(x)^2 + 5/9

$\Sigma$, to order $(\pi/f)^4$.

In [11]:
dS = diff(S, x) # nabla_mu Sigma
dSct = diff(S.C.T, x)
COM = v*S - S*v # [v_mu, Sigma]

using $[v_\mu, \Sigma]^\dagger = - [v_\mu, \Sigma^\dagger] $

In [12]:
term1 = mat_prep(dS*(dSct))
term2 = -I*mat_prep(dS*(-COM.C.T) + COM*dSct)
term3 = mat_prep(COM*COM.C.T)


var("dm", latex_name="\\Delta m", domain="real")
var("mm", latex_name="\\bar m", domain="real")
var("B0", latex_name="B_0", domain="real")

chi = (mm^2 * one + dm^2 * s3)

term4 = mat_prep(chi*S.C.T + S*chi.C.T, n=5)

terms_full = [term1, term2, term3, term4]

### All terms in the leading order Lagrangian
Sorted by powers of $\pi$. $f$ is not included

In [13]:
terms = [t.trace().full_simplify() for t in terms_full]

for i, t in enumerate(terms):
    print("term {}:".format(i+1))
    print_e(t)

term 1:
e^2 :


2*diff(pi1(x), x)^2 + 2*diff(pi2(x), x)^2 + 2*diff(pi3(x), x)^2

e^4 :


-2/3*pi2(x)^2*diff(pi1(x), x)^2 - 2/3*pi3(x)^2*diff(pi1(x), x)^2 + 4/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 2/3*pi1(x)^2*diff(pi2(x), x)^2 - 2/3*pi3(x)^2*diff(pi2(x), x)^2 + 4/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 4/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 2/3*pi1(x)^2*diff(pi3(x), x)^2 - 2/3*pi2(x)^2*diff(pi3(x), x)^2

term 2:
e :


...

-2*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*mm^2*cos(a)

e^3 :


2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*mm^2*pi1(x)*sin(a)

e^4 :


1/6*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)^2*mm^2*cos(a)

In [14]:
var("m", domain="real")
var("f", domain="real")

L_LO = 1/4*(term1+term2+term3+m^2*term4).trace().full_simplify().trig_reduce()
L_coeff = L_LO.coefficients(e)
L_coeff = [c[0].full_simplify().trig_reduce() for c in L_coeff]

# Leading order Lagrangian
$\mathcal{L}_4$ to $\mathcal{O}(\varepsilon^4)$, sorted by powers of $\varepsilon$ and $\mu_I$.

In [15]:
print_e(L_LO)

1 :


m^2*mm^2*cos(a)

1/2*d^2*mu^2*sin(a)^2

e :


-m^2*mm^2*pi1(x)*sin(a)

...

-2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2*cos(a)*pi1(x)*sin(a)

e^4 :


1/24*m^2*mm^2*cos(a)*pi1(x)^4 + 1/12*m^2*mm^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*mm^2*cos(a)*pi2(x)^4 + 1/12*m^2*mm^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*mm^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*mm^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

1/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*(pi2(x)*diff(pi1(x), x) - pi1(x)*diff(pi2(x), x))*d*mu*cos(a)

1/6*(2*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - pi1(x)^2 - pi2(x)^2)*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2

Comparing to more compact notation

In [16]:
(((p*p)**2*cos(a)/4*m**2 - ((p*p)*(diff(p, x)*diff(p, x) ) - (diff(p, x)*p)**2))/6).expand()

1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

### Next to leading order terms
Expanded to $\mathcal O (\varepsilon^3)$

In [17]:
S = SIGMA(3)
S = mat_prep(S, n=3)

In [18]:
var("xmu", latex_name="x_\\mu")
var("xnu", latex_name="x_\\nu")

chiSigmaMin = mat_prep(chi*S.C.T - S*chi.C.T, n=3)

$\chi \Sigma^\dagger -\Sigma \chi^\dagger$

In [19]:
res = (mat_prep(chiSigmaMin, n=3).trace()**2).series(e, 3).truncate()
print_e(res)

e^2 :


-16*dm^4*pi3(x)^2

$\chi \Sigma^\dagger + \Sigma \chi^\dagger$

In [20]:
chiSigmaPlus = mat_prep(chi*S.C.T + S*chi.T.C, n=3)
[pretty_print(t.factor()) for t in proj(chiSigmaPlus)];
chiSigmaPlus.trace()

-(e^2*cos(a)*pi1(x)^2 + e^2*cos(a)*pi2(x)^2 + e^2*cos(a)*pi3(x)^2 + 2*e*pi1(x)*sin(a) - 2*cos(a))*mm^2

-2*dm^2*e*pi2(x)

-(e^2*pi1(x)^2*sin(a) + e^2*pi2(x)^2*sin(a) + e^2*pi3(x)^2*sin(a) - 2*e*cos(a)*pi1(x) - 2*sin(a))*dm^2

-(e^2*cos(a)*pi1(x)^2 + e^2*cos(a)*pi2(x)^2 + e^2*cos(a)*pi3(x)^2 + 2*e*pi1(x)*sin(a) - 2*cos(a))*dm^2

-2*e^2*mm^2*cos(a)*pi1(x)^2 - 2*e^2*mm^2*cos(a)*pi2(x)^2 - 2*e^2*mm^2*cos(a)*pi3(x)^2 - 4*e*mm^2*pi1(x)*sin(a) + 4*mm^2*cos(a)

### Kinetic terms

$\nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger$

In [ ]:
S2 = mat_prep(SIGMA(3), n=3)
S2ct = S2.C.T
dS = diff(S2, x)

COM = v*S2 - S2*v
term1 = mat_prep(dS*(dSct))
term2 = -I*mat_prep(COM*dSct - dS*(COM.C.T))
term3 = mat_prep(COM*COM.C.T)

In [ ]:
print_e(term1.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term2.trace().series(e, 3).truncate().full_simplify().factor())
print_e(term3.trace().series(e, 3).truncate().full_simplify().factor())

kin = term1 + term2 + term3

Hack needed to get answer with free indices. I don'ẗ know any other way. This seems to work but be carefule, esp. with conjugate.

In [ ]:
dp = diff(p, x); dp
dpmu = vector([
    function(
        "pimu"+str(i), 
        latex_name="\\partial_\\mu \\pi_"+str(i), 
        conjugate_func=Id)(x) for i  in range(1, 4)
])

dpnu = vector([
    function(
        "pinu"+str(i), 
        latex_name="\\partial_\\nu \\pi_"+str(i), 
        conjugate_func=Id)(x) for i  in range(1, 4)
])

dprho = vector([
    function(
        "pirho"+str(i), 
        latex_name="\\partial_\\rho \\pi_"+str(i), 
        conjugate_func=Id)(x) for i  in range(1, 4)
])

In [ ]:
dSct = diff(S2ct, x)

dSmu = copy(dS)
dSrho = copy(dS)
dSnuct = copy(dSct)
dSmuct = copy(dSct)
dSrhoct = copy(dSct)
for i, dpi in enumerate(dp):
    dSmu = dSmu.subs(dpi==dpmu[i])
    dSrho = dSrho.subs(dpi==dprho[i])
    dSnuct = dSnuct.subs(dpi==dpnu[i])
    dSmuct = dSmuct.subs(dpi==dpmu[i])
    dSrhoct = dSrhoct.subs(dpi==dprho[i])

In [ ]:
var("dmu", latex_name="\\delta_\\mu", domain="real")
var("dnu", latex_name="\\delta_\\nu", domain="real")
var("drho", latex_name="\\delta_\\rho", domain="real")

vmu = 1/2*dmu*mu*s3
vnu = 1/2*dnu*mu*s3
vrho = 1/2*drho*mu*s3

COMmu = -(vmu*S2 - S2*vmu)      # [v_mu, Sigma]
COMmuct = vmu*S2ct - S2ct*vmu  # [v_nu, Sigma^\dagger]

COMnu = vnu*S2ct - S2ct*vnu  # [v_nu, Sigma^\dagger]
COMrho = -(vrho*S2 - S2*vrho)      # [v_mu, Sigma]
COMrhoct = vrho*S2ct - S2ct*vrho      # [v_mu, Sigma]

In [ ]:
term1a = mat_prep(dSmu*(dSmuct), n=3)
term2a = -I*mat_prep(dSmu*COMmuct - COMmu*dSmuct, n=3)
term3a = mat_prep(COMmu*COMmuct, n=3)
kin_mumu = term1a + term2a + term3a

term1 = mat_prep(dSrho*(dSrhoct), n=3)
term2 = -I*mat_prep(dSrho*COMrhoct - COMrho*dSrhoct, n=3)
term3 = mat_prep(COMrho*COMrhoct, n=3)
kin_rhorho = term1 + term2 + term3

term1 = mat_prep(dSmu*(dSnuct), n=3)
term2 = -I*mat_prep(dSmu*COMnu - COMmu*dSnuct, n=3)
term3 = mat_prep(COMmu*COMnu, n=3)
kin_munu = term1 + term2 + term3

# Constitutive terms for NLO Lagrangian

term_l1:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla^\mu \Sigma^\dagger \}^2$

In [ ]:
term_l1 = (kin_mumu.trace()*kin_rhorho.trace()).series(e, 3).truncate()
print_e(term_l1)

term_l2:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \nabla^\mu \Sigma \nabla^\nu \Sigma^\dagger \}$

In [ ]:
term_l2 = (kin_munu.trace()**2).series(e, 3).truncate()
print_e(term_l2)

term_l3

$\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}^2$

In [ ]:
term_l3 = (chiSigmaPlus.trace()**2).series(e, 3).truncate().factor()
print_e(term_l3)

term_l4

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}$

In [ ]:
term_l4 = (kin.trace()*chiSigmaPlus.trace()).series(e, 3).truncate()
print_e(term_l4)

term_l7

$\mathrm{Tr}\{ \chi \Sigma^\dagger - \Sigma \chi^\dagger \}^2$

In [ ]:
term_l7 = (mat_prep(chiSigmaMin, n=3).trace()**2).series(e, 3).truncate()
print_e(term_l7)

term chisq

$\mathrm{Tr}\{\chi^\dagger \chi\}$

In [ ]:
term_chisq = (chi*chi).trace().series(e, 3).truncate()
print_e(term_chisq)

last_term

$\mathrm{Tr}\{ (\chi \Sigma^\dagger)^2 + (\Sigma \chi^\dagger)^2 \}$

In [ ]:
term_last = ((chi*S2ct)**2 + (S2*chi)**2).trace().series(e, 3)
print_e(term_last)

### Names

term_l1:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla^\mu \Sigma^\dagger \}^2$

term_l2:

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \nabla^\mu \Sigma \nabla^\nu \Sigma^\dagger \}$

term_l3

$\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}^2$

term_l4

$\mathrm{Tr}\{ \nabla_\mu \Sigma \nabla_\nu \Sigma^\dagger \}
\mathrm{Tr}\{ \chi \Sigma^\dagger + \Sigma \chi^\dagger \}$

term_l7

$\mathrm{Tr}\{ \chi \Sigma^\dagger - \Sigma \chi^\dagger \}^2$

term chisq

$\mathrm{Tr}\{\chi^\dagger \chi\}$

term_last

$\mathrm{Tr}\{ (\chi \Sigma^\dagger)^2 + (\Sigma \chi^\dagger)^2 \}$

## NLO Lagrangian

In [ ]:
ls = [var("l{}".format(i), latex_name = "l_{}".format(i)) for i in range(1, 8)]
hs = [var("h{}".format(i), latex_name = "h_{}".format(i)) for i in range(1, 8)]

In [ ]:
LNLO = (
    l1 / 4 * term_l1
    + l2 / 4 * term_l2
    + (l3 + l4) / 16 * term_l3
    + l4 / 8 * term_l4
    - l7 / 16 * term_l7
    + (h1 + h3 - l4)/4 * term_chisq
    + (h1 - h3 - l4) / 16 * (term_l3 + term_l7 - 2*term_last )
)

In [ ]:
print_e(LNLO)